In [1]:
import os 
import tensorflow as tf
import importlib
import auxiliaryfunctions
# importlib.reload(auxiliaryfunctions)
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np
from scipy.stats import zscore,norm
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os
import pandas as pd
import random as python_random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,make_scorer,mean_absolute_error,r2_score

from datagenerator import dataGenerator
from keras import backend as K

from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, SGD


Bad key savefig.frameon in file /home/ubuntu/anaconda3/envs/deep_structure/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /home/ubuntu/anaconda3/envs/deep_structure/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /home/ubuntu/anaconda3/envs/deep_structure/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filen

In [2]:
np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

# lr=0.01
# decayRate = 0.001
early_stopping = True
epochs = 1000
batch_size = 4
use_dynamic_LR=False
# optimizer='Adam'
patience = 50
result_path = '/home/ubuntu/hacking/projects/deep-mediation/nov2020/results/simulation-3-results'
num_subs = [1000]
num_iters = 20
num_runs = 100
new_shape = [64,64]

In [5]:
def make_plot(z,m,d,iter):
    plt.figure(figsize=(20,5))
    plt.subplot(1, 4, 1)
    plot_distribution(z,m,labels=['z','m'])

    plt.subplot(1, 4, 2)
    plot_distribution(z,d,labels=['z','d'])

    plt.subplot(1, 4, 3)
    plot_scatter(z,d,labels= ['z','d'])

    plt.subplot(1, 4, 4)
    plot_scatter(z,m,labels= ['z','m'])
    # plt.savefig(os.path.join(result_path,'result'+ str(iter) +'.png'),dpi=300)
    plt.show()

def plot_distribution(z,m,labels):
    sns.distplot(z, hist=True, rug=True,label=labels[0]);
    sns.distplot(m, hist=True, rug=True,label=labels[1]);
    plt.legend()

def plot_scatter(y_test,y_pred,labels):
    r = np.corrcoef(y_pred,y_test)[0,1]
    r = round(r,3)
    plt.scatter(y_test,y_pred)
    plt.xlabel(labels[0])
    plt.ylabel(labels[1])
    plt.title('r=%s'%r)

def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('MSE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

def plot_error(ax,values,title,y_lab,num_iterations):
    ax.plot(values)
    ax.set_xlim([0,num_iterations])
    ax.set_xlabel('#iterations')
    ax.set_ylabel(y_lab)
    ax.set_title(title)

def create_empty_df(num_runs,num_iters):
    # Creates an empty dataFrame
    a = np.empty((num_runs,num_iters))
    a[:] = np.nan
    dataFrame = None
    parameters = ['alpha0', 'beta0','alpha','beta','gamma']
    for params in parameters:
        iter = ['iter_'+str(i) for i in range(num_iters)]
        pdindex = pd.MultiIndex.from_product([[params], iter],
                                             names=['parameters', 'runs']) 
        frame = pd.DataFrame(a, columns = pdindex,index = range(0,num_runs))
        dataFrame = pd.concat([dataFrame,frame],axis=1)
    return dataFrame

def simulate_mediation(df,M,params_df,model,algo,num_subs,batch_size,epochs,tune,n,n_runs,iterations):
    """
    """
    early_stopping=True
    # Initialize z with some phi
    print("Using intial random model parameters")
    z = model.predict(M)#at this point, the model is untrained and its weights are initialied using He initializer
    print(z[0:5])
    try:
        z = np.concatenate(z)
    except:
        pass
    
    for i in range(0,iterations):
        print('Starting iteration... %s'%(i+1))
        #Check for sign change
        print(z[0:5])
        if np.corrcoef(z,df.Y)[0,1] < 0 :
            z = z*(-1)
        # Check for scaling
        z= zscore(z)
        lm = smf.ols(formula='z ~ X', data=df).fit()
        alpha0 = lm.params.loc['Intercept'] 
        alph = lm.params.loc['X'] 

        lm = smf.ols(formula='Y ~ z + X', data=df).fit()
        beta0 = lm.params.Intercept 
        bet = lm.params.z
        gam = lm.params.X 
        resid_std = np.std(lm.resid)

        e = df.Y - beta0 - (df.X*gam)
        h = alpha0 + df.X*alph
        d = (((bet*e)+h)/((bet**2)+1))
#         d = zscore(d)
        if tune:
              if algo == 'cnn':
                  
                if early_stopping:
                    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=20)
                    history = model.fit(M,d,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=[es],
                                      shuffle=True,validation_split = 0.3)
                else:
                    history = model.fit(M,d,batch_size=batch_size,epochs=epochs,verbose=1,
                                  shuffle=True,validation_split = 0.3)

        z = model.predict(M)

        params_df.loc[n_runs]['alpha0','iter_'+str(i)]=alpha0
        params_df.loc[n_runs]['beta0','iter_'+str(i)]=beta0
        params_df.loc[n_runs]['beta','iter_'+str(i)]=bet
        params_df.loc[n_runs]['gamma','iter_'+str(i)]=gam
        params_df.loc[n_runs]['alpha','iter_'+str(i)]=alph

        try:
            z = np.concatenate(z)
        except:
            pass        
    return params_df,z


In [14]:
smf.ols?

In [28]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

for n in num_subs:
    start = 0
    output_df = create_empty_df(num_runs,num_iters)
    for runs in range(start,num_runs):
        K.clear_session()
        print("Running simulation for number of subjects %s starting with %s runs"%(n,runs+1))
    # simulate the dataset
        df,mediator,z = auxiliaryfunctions.simulate_dataset(n,resize= True,
                                          new_shape=new_shape,cnn=True,visualize=False,alpha=0.2)
    # create a model 
        input_shape = (new_shape[0],new_shape[1]*4,1)
        model = auxiliaryfunctions.create_model(mediator,df.m,input_shape=input_shape)

        batch_size = 4
        algo = 'cnn'
        params_df,z_final = simulate_mediation(df,mediator,output_df,
                                      model,algo,n,batch_size,epochs,
                                      True,n,runs,num_iters)
        params_df.to_excel(os.path.join(result_path,'simulation-3-beta-0-num-subjs-'+str(n)+'-part-4.xlsx'))

Running simulation for number of subjects 1000 starting with 1 runs
Using intial random model parameters
(1000, 1)
[[-13.996438]
 [-26.762455]
 [-18.874699]
 [-16.4507  ]
 [-29.906397]]
Starting iteration... 1
[-13.996438 -26.762455 -18.874699 -16.4507   -29.906397]
Epoch 1/1000
175/175 [==============================] - 2s 10ms/step - loss: 10320.7031 - mse: 10320.7031 - val_loss: 2.1668 - val_mse: 2.1668
Epoch 2/1000
175/175 [==============================] - 2s 9ms/step - loss: 3.2725 - mse: 3.2725 - val_loss: 0.3488 - val_mse: 0.3488
Epoch 3/1000
175/175 [==============================] - 1s 6ms/step - loss: 1.1216 - mse: 1.1216 - val_loss: 0.1587 - val_mse: 0.1587
Epoch 4/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.5860 - mse: 0.5860 - val_loss: 0.0697 - val_mse: 0.0697
Epoch 5/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.3444 - mse: 0.3444 - val_loss: 0.0447 - val_mse: 0.0447
Epoch 6/1000
175/175 [==============================] 

Epoch 60/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.0044 - val_mse: 0.0044
Epoch 61/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.0044 - val_mse: 0.0044
Epoch 62/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.0029 - mse: 0.0029 - val_loss: 0.0045 - val_mse: 0.0045
Epoch 63/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.0044 - val_mse: 0.0044
Epoch 64/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.0044 - val_mse: 0.0044
Epoch 65/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0047 - val_mse: 0.0047
Epoch 66/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0048 - val_mse: 0.0048
Epoch 67/1000
175/175 [==============================] 

175/175 [==============================] - 2s 9ms/step - loss: 0.2102 - mse: 0.2102 - val_loss: 0.4069 - val_mse: 0.4069
Epoch 6/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.2046 - mse: 0.2046 - val_loss: 0.4422 - val_mse: 0.4422
Epoch 7/1000
175/175 [==============================] - 1s 7ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.4505 - val_mse: 0.4505
Epoch 8/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.2136 - mse: 0.2136 - val_loss: 0.4344 - val_mse: 0.4344
Epoch 9/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1929 - mse: 0.1929 - val_loss: 0.4381 - val_mse: 0.4381
Epoch 10/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.2292 - mse: 0.2292 - val_loss: 0.4415 - val_mse: 0.4415
Epoch 11/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.2064 - mse: 0.2064 - val_loss: 0.4303 - val_mse: 0.4303
Epoch 12/1000
175/175 [==============================] - 2s 9ms/step - lo

175/175 [==============================] - 2s 9ms/step - loss: 0.1947 - mse: 0.1947 - val_loss: 0.5217 - val_mse: 0.5217
Epoch 34/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.1783 - mse: 0.1783 - val_loss: 0.4926 - val_mse: 0.4926
Epoch 35/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.1981 - mse: 0.1981 - val_loss: 0.4902 - val_mse: 0.4902
Epoch 36/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.2196 - mse: 0.2196 - val_loss: 0.5484 - val_mse: 0.5484
Epoch 37/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1973 - mse: 0.1973 - val_loss: 0.5068 - val_mse: 0.5068
Epoch 38/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1892 - mse: 0.1892 - val_loss: 0.5316 - val_mse: 0.5316
Epoch 39/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.2062 - mse: 0.2062 - val_loss: 0.4779 - val_mse: 0.4779
Epoch 40/1000
175/175 [==============================] - 1s 8ms/step 

175/175 [==============================] - 2s 9ms/step - loss: 0.2112 - mse: 0.2112 - val_loss: 0.6203 - val_mse: 0.6203
Epoch 10/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.2307 - mse: 0.2307 - val_loss: 0.6393 - val_mse: 0.6393
Epoch 11/1000
175/175 [==============================] - 1s 7ms/step - loss: 0.2150 - mse: 0.2150 - val_loss: 0.5861 - val_mse: 0.5861
Epoch 12/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.2331 - mse: 0.2331 - val_loss: 0.6444 - val_mse: 0.6444
Epoch 13/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.2314 - mse: 0.2314 - val_loss: 0.6298 - val_mse: 0.6298
Epoch 14/1000
175/175 [==============================] - 1s 6ms/step - loss: 0.2048 - mse: 0.2048 - val_loss: 0.6471 - val_mse: 0.6471
Epoch 15/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.2114 - mse: 0.2114 - val_loss: 0.6510 - val_mse: 0.6510
Epoch 16/1000
175/175 [==============================] - 2s 9ms/step 

...

175/175 [==============================] - 2s 9ms/step - loss: 0.2093 - mse: 0.2093 - val_loss: 0.6708 - val_mse: 0.6708
Epoch 18/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.1815 - mse: 0.1815 - val_loss: 0.6913 - val_mse: 0.6913
Epoch 19/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.1956 - mse: 0.1956 - val_loss: 0.6673 - val_mse: 0.6673
Epoch 20/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.2050 - mse: 0.2050 - val_loss: 0.6564 - val_mse: 0.6564
Epoch 21/1000
175/175 [==============================] - 1s 6ms/step - loss: 0.2157 - mse: 0.2157 - val_loss: 0.6979 - val_mse: 0.6979
Epoch 22/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.2015 - mse: 0.2015 - val_loss: 0.6574 - val_mse: 0.6574
Epoch 23/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1974 - mse: 0.1974 - val_loss: 0.6407 - val_mse: 0.6407
Epoch 24/1000
175/175 [==============================] - 1s 7ms/step 

175/175 [==============================] - 1s 5ms/step - loss: 0.2034 - mse: 0.2034 - val_loss: 0.6793 - val_mse: 0.6793
Epoch 18/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1849 - mse: 0.1849 - val_loss: 0.6300 - val_mse: 0.6300
Epoch 19/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.1969 - mse: 0.1969 - val_loss: 0.6591 - val_mse: 0.6591
Epoch 20/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.1932 - mse: 0.1932 - val_loss: 0.6315 - val_mse: 0.6315
Epoch 21/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.1932 - mse: 0.1932 - val_loss: 0.6497 - val_mse: 0.6497
Epoch 22/1000
175/175 [==============================] - 1s 9ms/step - loss: 0.1878 - mse: 0.1878 - val_loss: 0.6503 - val_mse: 0.6503
Epoch 23/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.1816 - mse: 0.1816 - val_loss: 0.6477 - val_mse: 0.6477
Epoch 24/1000
175/175 [==============================] - 1s 9ms/step 

175/175 [==============================] - 1s 9ms/step - loss: 0.1852 - mse: 0.1852 - val_loss: 0.6437 - val_mse: 0.6437
Epoch 14/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.1793 - mse: 0.1793 - val_loss: 0.6254 - val_mse: 0.6254
Epoch 15/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.2053 - mse: 0.2053 - val_loss: 0.6702 - val_mse: 0.6702
Epoch 16/1000
175/175 [==============================] - 1s 6ms/step - loss: 0.1811 - mse: 0.1811 - val_loss: 0.6623 - val_mse: 0.6623
Epoch 17/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1795 - mse: 0.1795 - val_loss: 0.6587 - val_mse: 0.6587
Epoch 18/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1938 - mse: 0.1938 - val_loss: 0.6533 - val_mse: 0.6533
Epoch 19/1000
175/175 [==============================] - 1s 7ms/step - loss: 0.1861 - mse: 0.1861 - val_loss: 0.6878 - val_mse: 0.6878
Epoch 20/1000
175/175 [==============================] - 1s 8ms/step 

175/175 [==============================] - 2s 9ms/step - loss: 0.1903 - mse: 0.1903 - val_loss: 0.6733 - val_mse: 0.6733
Epoch 25/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.2046 - mse: 0.2046 - val_loss: 0.6553 - val_mse: 0.6553
Epoch 26/1000
175/175 [==============================] - 1s 7ms/step - loss: 0.2048 - mse: 0.2048 - val_loss: 0.6606 - val_mse: 0.6606
Epoch 27/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1982 - mse: 0.1982 - val_loss: 0.6540 - val_mse: 0.6540
Epoch 28/1000
175/175 [==============================] - 1s 5ms/step - loss: 0.1799 - mse: 0.1799 - val_loss: 0.6781 - val_mse: 0.6781
Epoch 00028: early stopping
Starting iteration... 20
[ 0.5869232  -0.08848982 -0.39731932 -0.8733503  -0.3350648 ]
Epoch 1/1000
175/175 [==============================] - 1s 8ms/step - loss: 0.1961 - mse: 0.1961 - val_loss: 0.6747 - val_mse: 0.6747
Epoch 2/1000
175/175 [==============================] - 2s 9ms/step - loss: 0.2006 - mse: 

KeyboardInterrupt: 

In [18]:
runs

2

In [27]:
params_df['beta'].head(15)

runs,iter_0,iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8,iter_9,iter_10,iter_11,iter_12,iter_13,iter_14,iter_15,iter_16,iter_17,iter_18,iter_19
0,0.032916,0.235453,0.295314,0.301690,0.207344,0.304704,0.305745,0.305594,0.300936,0.282327,0.29586,0.285591,0.299935,0.283717,0.285412,0.268729,0.283532,0.307701,0.308587,0.314204
1,0.024394,0.117174,0.092676,0.100982,0.120006,0.157552,0.223433,0.223217,0.261170,0.283378,0.29567,0.300617,0.288650,0.306214,0.333991,0.351387,0.363177,0.348122,0.359462,0.365071
2,0.025716,0.116223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
!nvidia-smi

Wed Dec  2 03:13:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    38W / 300W |    501MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!kill -9 3697

In [ ]:
z[0:5]